In [3]:
import pandas as pd
import numpy as np
final_data_df = pd.read_csv('final_data.csv')

# Filter out participants who according to their anwser to certain questions
# and therefore are not eligible for the study
final_data_df = final_data_df[final_data_df['DataProtectionAgreement'] == True]
final_data_df = final_data_df[final_data_df['LiabilityWaiverSigned'] == True]
final_data_df = final_data_df[final_data_df['HasPacemaker'] == False]
final_data_df = final_data_df[final_data_df['Colorblindness'] == False]
final_data_df = final_data_df[final_data_df['HandDominance'] == 'right']

# Get the UEQ columns grouped by the condition column
ueq_columns = [col for col in final_data_df.columns if col.startswith(
    'UEQ')] + ['Condition']


# Filter the DataFrame to only include the relevant columns
filtered_df = final_data_df[ueq_columns]
filtered_df.head()
# Print all rows with condition equals control
#filtered_df[filtered_df['Condition'] == 'Control'].drop(columns=['Condition']).to_csv('ueq_control.csv', index=False)
#filtered_df[filtered_df['Condition'] == 'Visual'].drop(columns=['Condition']).to_csv('ueq_visual.csv', index=False)
#filtered_df[filtered_df['Condition'] == 'EMS'].drop(columns=['Condition']).to_csv('ueq_ems.csv', index=False)
#filtered_df[filtered_df['Condition'] == 'EMSVisual'].drop(columns=['Condition']).to_csv('ueq_emsvisual.csv', index=False)


,UEQ_behindernd_unterstützend,UEQ_kompliziert_einfach,UEQ_ineffizient_effizient,UEQ_verwirrend_übersichtlich,UEQ_langweilig_spannend,UEQ_uninteressant_interessant,UEQ_konventionell_originell,UEQ_herkömmlich_neuartig,Condition
0,4,4,3,1,7,7,6,6,Control
1,6,3,6,4,7,7,7,6,Visual
2,6,3,6,7,7,7,7,7,EMS
3,7,5,7,5,7,7,7,7,EMSVisual
4,5,6,6,2,6,6,5,5,Visual


In [6]:
from scipy.stats import sem

def mean_if_count_greater_than_zero(columns):
    values = columns.dropna().astype(float)
    return values.mean() if len(values) > 0 else ""

def process_condition(df, condition):
    filtered_df = df[df['Condition'] == condition]

    # Remove the condition column
    filtered_df = filtered_df.drop(columns='Condition')

    filtered_df = filtered_df.applymap(lambda x: x-4 if x > 0 else "")

    filtered_df['Pragmatic Quality'] = filtered_df.iloc[:, 0:4].apply(mean_if_count_greater_than_zero, axis=1)
    filtered_df['Hedonic Quality'] = filtered_df.iloc[:, 4:8].apply(mean_if_count_greater_than_zero, axis=1)
    filtered_df['Overall'] = filtered_df.iloc[:, 0:8].apply(mean_if_count_greater_than_zero, axis=1)

    def calculate_mean_sd_se(column):
        values = column.dropna().astype(float)
        mean = values.mean()
        sd = values.std()
        se = sem(values) if len(values) > 1 else None  # Requires at least 2 values to compute SE
        return mean, sd, se

    pragmatische_qualität, sd_prag, se_prag = calculate_mean_sd_se(
        filtered_df['Pragmatic Quality'])
    hedonische_qualität, sd_hed, se_hed = calculate_mean_sd_se(
        filtered_df['Hedonic Quality'])
    overall, sd_over, se_over = calculate_mean_sd_se(filtered_df['Overall'])

    return pd.DataFrame({
        'Condition': [condition],
        'Pragmatic Quality': [pragmatische_qualität],
        'SD Pragmatic Quality': [sd_prag],
        'SE Pragmatic Quality': [se_prag],
        'Hedonic Quality': [hedonische_qualität],
        'SD Hedonic Quality': [sd_hed],
        'SE Hedonic Quality': [se_hed],
        'Overall': [overall],
        'SD Overall': [sd_over],
        'SE Overall': [se_over]
    })

conditions = ["Control","Visual","EMS","EMSVisual"]
result_df = pd.concat([process_condition(filtered_df, condition)
                      for condition in conditions]).reset_index(drop=True)
result_df.to_csv('ueq_final.csv', index=False)
result_df

,Condition,Pragmatic Quality,SD Pragmatic Quality,SE Pragmatic Quality,Hedonic Quality,SD Hedonic Quality,SE Hedonic Quality,Overall,SD Overall,SE Overall
0,Control,1.178571,1.128429,0.301585,0.482143,1.639255,0.438109,0.830357,0.774608,0.207023
1,Visual,0.821429,1.432549,0.382865,1.553571,1.047793,0.280035,1.187500,1.043236,0.278817
2,EMS,0.678571,1.422446,0.380165,2.035714,0.848399,0.226744,1.357143,0.918372,0.245445
3,EMSVisual,0.285714,1.423893,0.380552,2.267857,0.710256,0.189824,1.276786,0.893592,0.238822
